# Case 1: ParamType is a Reference or Pointer, but not a Universal Reference

Meyers, Scott. Effective Modern C++ . O'Reilly Media. Kindle Edition. 

##  Key idea: pinch of pseudocode
 
    If the function template looks like this:
 
      template <typename T>
      void f(ParamType param);
 
    then two types are deduced: one for T and one for ParamType.  These types
    are frequently different, because ParamType often contains adornments,
    e.g., const- or reference qualifiers.

In [1]:
#pragma cling add_include_path("/usr/local/include")
#include <boost/type_index.hpp>
#include <iostream>

In [2]:
template<typename T>
void f(const T& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}  // ParamType is const T&

In [3]:
{
  int x = 0;
  f(x);                    // call f with an int
}

T     = int
param = int const&


##  Key idea: non const
 
    Considering the general form for templates and calls to it:
 
      template <typename T>
      void f(ParamType param);
 
      f(expr);                // deduce T and ParamType from expr
 
    then, in the simplest case when ParamType is a reference type or a pointer
    type, but not a universal reference, type deduction works like this:
 
    1. If expr's type is a reference, ignore the reference part.
    2. Then pattern-match expr's type against ParamType to determine T.

In [4]:
template<typename T>
void f(T& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';
}   // param is a reference

In [5]:
{    
  int x = 27;         // x is an int
  const int cx = x;   // cx is a const int
  const int& rx = x;  // rx is a reference to x as a const int

  f(x);               // T is int, param's type is int&

  f(cx);              // T is const int,
                      // param's type is const int&

  f(rx);              // T is const int,
                      // param's type is const int&
}

T     = int
param = int&
T     = int
param = int const&
T     = int
param = int const&


##  Key idea: const
 
    Considering the general form for templates and calls to it:
 
      template <typename T>
      void f(ParamType param);
 
      f(expr);                // deduce T and ParamType from expr
 
    then, in the simplest case when ParamType is a pointer type or a reference
    type, but not a universal reference, type deduction works like this:
 
    1. If expr's type is a reference, ignore the reference part.
    2. Then pattern-match expr's type against ParamType to determine T.
 
    If the type of f's parameter is changed from T& to const T&, the constness
    of cx and rx continues to be respected, but because we're now assuming that
    param is a reference-to-const, there's no longer a need for const to be
    deduced as part of T.

In [6]:
template<typename T>
void f1(const T& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}  // param is now a ref-to-const

In [7]:
{    
  int x = 27;              // as before
  const int cx = x;        // as before
  const int& rx = x;       // as before

  f1(x);                    // T is int, param's type is const int&

  f1(cx);                   // T is int, param's type is const int&

  f1(rx);                   // T is int, param's type is const int&
}

T     = int
param = int const&
T     = int
param = int const&
T     = int
param = int const&


##  Key idea: pointer
 
    Considering the general form for templates and calls to it:
 
      template <typename T>
      void f(ParamType param);
 
      f(expr);                // deduce T and ParamType from expr
 
    then, in the simplest case when ParamType is a pointer type or a reference
    type, but not a universal reference, type deduction works like this:
 
    1. If expr's type is a reference, ignore the reference part.
    2. Pattern-match expr's type against ParamType to determine T.
 
    If param were a pointer (or a pointer to const) instead of a reference,
    things would work essentially the same way:

In [8]:
template<typename T>
void f2(T* param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

};      // param is now a pointer

In [9]:
{
  int x = 27;            // as before
  const int *px = &x;    // px is a ptr to x as a const int

  f2(&x);                 // T is int, param's type is int*

  f2(px);                 // T is const int, param's type is const int*
}

T     = int
param = int*
T     = int const
param = int const*


# Case 2: ParamType is a Universal Reference

Meyers, Scott. Effective Modern C++ . O'Reilly Media. Kindle Edition. 

##  Key idea: uref
 
    Considering the general form for templates and calls to it:
 
      template <typename T>
      void f(ParamType param);
 
      f(expr);                // deduce T and ParamType from expr
 
    then, in the case when ParamType is a universal reference
    type, type deduction works like this:
 
    1. If expr is an lvalue, both T and ParamType are deduced to be lvalue
      references
    2. If expr is an rvalue, the usual type deduction rules apply.


In [10]:
template<typename T>
void f3(T&& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}      // param is now a universal reference

In [11]:
{
  int x = 27;             // as before
  const int cx = x;       // as before
  const int& rx = x;      // as before

  f3(x);                   // x is lvalue, so T is int&,
                          // param's type is also int&

  f3(cx);                  // cx is lvalue, so T is const int&,
                          // param's type is also const int&

  f3(rx);                  // rx is lvalue, so T is const int&,
                          // param's type is also const int&

  f3(27);                  // 27 is rvalue, so T is int,
                          // param's type is therefore int&&
}

T     = int&
param = int&
T     = int const&
param = int const&
T     = int const&
param = int const&
T     = int
param = int&&


# Case 3: ParamType is Neither a Pointer nor a Reference

Meyers, Scott. Effective Modern C++ . O'Reilly Media. Kindle Edition. 

##  Key idea: pass by value
 
    If we're dealing with pass-by-value
 
      template <typename T>
      void f(T param);        // param is now passed by value
 
    That means that param will be a copy of whatever is passed in - a
    completely new object.  The fact that param will be a new object motivates
    the rules that govern how T is deduced from expr:
 
    1. As before, if expr's type is a reference, ignore the reference part.
 
    2. If, after ignoring expr's reference-ness, expr is const, ignore that,
       too.  If it's volatile, also ignore that. (volatile objects are uncommon.
       They're generally used only for implementing device drivers.)


In [12]:
template<typename T>
void f4(T param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}          // param is now passed by value

In [13]:
{
  int x = 27;               // as before
  const int cx = x;         // as before
  const int& rx = x;        // as before
  f4(x);                     // T's and param's types are both int

  f4(cx);                    // T's and param's types are again both int

  f4(rx);                    // T's and param's types are still both int


  const char* const ptr =   // ptr is const pointer to const object
    "Fun with pointers";

  f(ptr);                   // pass arg of type const char * const
}

T     = int
param = int
T     = int
param = int
T     = int
param = int
T     = char const*
param = char const* const&


# Array Arguments

Meyers, Scott. Effective Modern C++ . O'Reilly Media. Kindle Edition. 

##  Key idea: array to pointer decay rule
 
    In many contexts, an array decays into a pointer to its first element.
 

In [14]:
{
    const char name[] = "J. P. Briggs";  // name's type is
                                         // const char[13]

    const char * ptrToName = name;       // array decays to pointer
}

##  Key idea: arrays by value
 
    Because array parameter declarations are treated as if they were pointer
    parameters, the type of an array that's passed to a template function by
    value is deduced to be a pointer type.


In [15]:
template<typename T>
void f5(T param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}  // template with by-value parameter

In [16]:
{
    const char name[] = "J. P. Briggs";  // name's type is
                                         // const char[13]

    f5(name);                   // what types are deduced for T and param?
                               // -> name is array, but T deduced as const char*
}

T     = char const*
param = char const*


## Key idea: array and pointer parameter equivalence

In [17]:
void myFunc1(int param[]) {}

In [18]:
void myFunc2(int* param) {}   // same function as above

##  Key idea: arrays by reference
 
    Although functions can't declare parameters that are truly arrays, they can
    declare parameters that are references to arrays.
 
    The type deduced for T is the actual type of the array!  That type includes
    the size of the array, so in this example T is deduced to be const
    char[13], and the type of f's parameter (a reference to this array) is
    const char (&)[13].

In [19]:
template<typename T>
void f6(T& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}  // template with by-reference parameter

In [20]:
{
    const char name[] = "J. P. Briggs";  // name's type is
                                         // const char[13]

    f6(name);                   // pass array to f
}

T     = char const [13]
param = char const (&) [13]


##  Key idea: deduce number of array elements
 
    The ability to declare references to arrays enables creation of a template
    to deduce the number of elements that an array contains.

In [21]:
#include <array>
#include <cstddef>

// return size of an array as a compile-time constant. (The
// array parameter has no name, because we care only about
// the number of elements it contains.)
template<typename T, std::size_t N>                 // see info
constexpr std::size_t arraySize(T (&)[N]) noexcept  // below on
{                                                   // constexpr 
    return N;                                       // and
}                                                   // noexcept


int keyVals[] = { 1, 3, 7, 9, 11, 22, 35 };  // keyVals has
                                             // 7 elements

int mappedVals1[arraySize(keyVals)];         // so does
                                             // mappedVals

std::array<int, arraySize(keyVals)> mappedVals2;  // mappedVals'
                                                  // size is 7

## Function Arguments

Meyers, Scott. Effective Modern C++ . O'Reilly Media. Kindle Edition. 

##  Key-idea: function to pointer decay rule
 
    Function types can decay into pointers, too, and everything regarding type
    deduction and arrays applies to type deduction for functions and their decay
    into function pointers.

In [22]:
void someFunc(int, double){}  // someFunc is a function;
                              // type is void(int, double)

In [23]:
template<typename T>
void g1(T param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}           // in f1, param passed by value

In [24]:
template<typename T>
void g2(T& param) {
    using std::cout;
    using boost::typeindex::type_id_with_cvr;

    cout << "T     = " << type_id_with_cvr<T>().pretty_name() << '\n';     
    cout << "param = " << type_id_with_cvr<decltype(param)>().pretty_name() << '\n';

}          // in f2, param passed by ref

In [25]:
{
  g1(someFunc);               // param deduced as ptr-to-func;
                              // type is void (*)(int, double)

  g2(someFunc);               // param deduced as ref-to-func;
                              // type is void (&)(int, double)
}

T     = void (*)(int, double)
param = void (*)(int, double)
T     = void (int, double)
param = void (&)(int, double)


                               Things to Remember
                               ------------------

* During template type deduction, arguments that are references are treated as
  non-references, i.e., their reference-ness is ignored.

* When deducing types for universal reference parameters, lvalue arguments get
  special treatment.

* When deducing types for by-value parameters, const and/or volatile arguments
  are treated as non-const and non-volatile.

* During template type deduction, arguments that are array or function names
  decay to pointers, unless they're used to initialize references.

